# Application of Transferable Adversarial Attacks on Convolutional Neuronal Networks: An Evaluation of Existing Attack and Defense Mechanisms

This code is beloinging to the bachelor thesis "Application of Transferable Adversarial Attacks on Convolutional Neural Networks: An Evaluation of Existing Attack and Defense Mechanisms". All results of this work can be validated and reproduced. The whole Process is divided into five phases:

* **Phase 0: Initialize Project**

* **Phase 1: Create Models for Evaluation**

* **Phase 2: Generate (Adversarial) Datasets**

* **Phase 3: Evaluate (Adversarial) Datasets on Models**

* **Phase 4: Generate Visualizations of Results**

To retrain models, the corresponding model must be deleted from the "trained_models/..." folder and phase 1 executed. The same applies to the evaluation datasets and phase 2.

## Dependencies

Import all required dependencies.

In [1]:
# Tensorflow, NumPy and Keras
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

# Enable memory growth in Tensorflow to prevent memory errors
physical_devices = tf.config.list_physical_devices("GPU")
for gpu_instance in physical_devices:
    tf.config.experimental.set_memory_growth(gpu_instance, True)

# Check if files already exist
from os.path import exists
from data import DataLoader

# Hide all warnings
import warnings
warnings.filterwarnings("ignore")

## Phase 0: Initialize Project

In [ ]:
print("--- Phase 0: Initialize Evaluation ---")

# only execute if the tiny ImageNet folder does not exist
if exists("tiny-imagenet-200")==False:
    print("Install requirements and retrieve dataset.")
    
    from init import InitProject
    
    # Install required packages and download tiny-imagenet-200 from Stanford repository
    initializer = InitProject()
    initializer.installRequirements()
    initializer.retrieveDataset()
    
    # Transform downloaded dataset to Tensorflow objects and save to:
    # "datasets/tiny_imagenet/{training|test|validation}"
    loader = DataLoader()
    loader.saveData()
else:
    print("Evaluation already initialized.")

## Phase 1: Create Models for Evaluation

Build and train all models that we need for our evaluation (time consuming without GPU).

In [ ]:
print("--- Phase 1: Create and train models ---")

# Load training and validation datasets
training_dataset = tf.data.experimental.load("datasets/tiny_imagenet/training/")
validation_dataset = tf.data.experimental.load("datasets/tiny_imagenet/validation/")

# 1. Base model

from training import BaseModelGenerator
baseGen = BaseModelGenerator()

# Generate a target base model
if exists("trained_models/target.h5")==False:
    print("Create target model.")
    baseGen.generateBaseModel(training_dataset,
                              validation_dataset,
                              name="target")
else:
    print("Target model exists.")


# Generate a substitute base model
if exists("trained_models/substitute.h5")==False:
    print("Create substitute model.")
    baseGen.generateBaseModel(training_dataset,
                              validation_dataset,
                              name="substitute")
else:
    print("Substitute model exists.")
    
# 2. Adversarial Training model

from training_AdversarialTraining import AdversarialTrainingGenerator
adversarialTrainingGen = AdversarialTrainingGenerator()

# Generate a adversarial training model
if exists("trained_models/defenses/adversarial_training/adversarial_training.h5")==False:
    print("Create Adversarial Training model.")
    adversarialTrainingGen.generateAdversarialTrainingModel(training_dataset, 
                                                        validation_dataset, 
                                                        alpha=0.5,
                                                        eps=16.0,
                                                        name="adversarial_training")
else:
    print("Adversarial Training model exists.")
    
if exists("trained_models/defenses/adversarial_training/adversarial_training_with_sampling.h5")==False:
    print("Create Adversarial Training model with sampling from U(0,32).")
    adversarialTrainingGen.generateAdversarialTrainingModel(training_dataset, 
                                                        validation_dataset, 
                                                        alpha=0.5,
                                                        eps_sample=True,
                                                        name="adversarial_training_with_sampling")
else:
    print("Adversarial Training model with sampling from U(0,32) exists.")
    
# 3. Madry model

from training_Madry import MadryGenerator
madryGen = MadryGenerator()

# Generate a madry defense model with eps=8.0, nb_iter=10
if exists("trained_models/defenses/madry/madry_e8_n10.h5")==False:
    print("Create Madry model with eps 8 and n 10.")
    madryGen.generateMadryModel(training_dataset, 
                                validation_dataset, 
                                alpha=0.5,
                                eps=8.0,
                                eps_iter=0.8,
                                nb_iter=10,
                                name="madry_e8_n10")
else:
    print("Madry defense model with eps 8 and n 10 exists.")

# Generate a madry defense model with eps=16.0, nb_iter=20
if exists("trained_models/defenses/madry/madry_e16_n20.h5")==False:
    print("Create Madry model with eps 16 and n 20.")
    madryGen.generateMadryModel(training_dataset, 
                                validation_dataset, 
                                alpha=0.5,
                                eps=16.0,
                                eps_iter=0.8,
                                nb_iter=20,
                                name="madry_e16_n20")
else:
    print("Madry defense model with eps 16 and n 20 exists.")
    
# 4. Seiler model

from training_Seiler import SeilerGenerator
superimposingGen = SeilerGenerator()

# Generate a superimposing defense model with lambda=0
if exists("trained_models/defenses/seiler/seiler_lambda0.h5")==False:
    print("Create Superimposing defense model with lambda 0.")
    superimposingGen.generateSeilerModel(training_dataset,
                                         validation_dataset,
                                         _lambda=0,
                                        name="seiler_lambda0")
else:
    print("Superimposing defense model with lambda 0 exists.")


# Generata a superimposing defense model with lambda=10
if exists("trained_models/defenses/seiler/seiler_lambda10.h5")==False:
    print("Create Superimposing defense model with lambda 10.")
    superimposingGen.generateSeilerModel(training_dataset,
                                         validation_dataset,
                                         _lambda=10,
                                         name="seiler_lambda10")
else:
    print("Superimposing defense model with lambda 10 exists.")

# 5. Defensive Destilation model

from training_DefensiveDistillation import DefensiveDistillationGenerator
ddGenerator = DefensiveDistillationGenerator()

# Generate a defensive destilation model with tau=50
if exists("trained_models/defenses/defensive_distillation/defensive_distillation.h5")==False:
    print("Create defensive destillation model with tau 50.")
    ddGenerator.generateDefensiveDistillationModel(training_dataset, 
                                                   validation_dataset,
                                                   train_temp=50,
                                                   epochs=100)
else:
    print("Defensive destillation model with tau 50 exists.")

## Phase 2: Generate (Adversarial) Datasets

Based on the subsitute, create all black-box datasets we need for the evaluation (time consuming without GPU).

In [ ]:
print("--- Phase 2: Generate adversarial datasets ---")

# load the subsitute we use for the creation of the datasets
model_substitute = tf.keras.models.load_model("trained_models/substitute.h5")


# Create 100% accuracy evalaution dataset based on testing dataset and substitute model
if exists("datasets/evaluation_clean/dataset_spec.pb")==False:
    print("Create clean evaluation dataset.")
    testing_dataset = tf.data.experimental.load("datasets/tiny_imagenet/testing/")
    loader = DataLoader()
    loader.generate_clean_evaluation_dataset(testing_dataset, model_substitute)
else:
    print("Clean evaluation dataset already exists.")

clean_dataset = tf.data.experimental.load("datasets/evaluation_clean/")

# 1. Generate adversarial datasts with momentum attack

from attack_momentum import MomentumAttack
MI_FGSM = MomentumAttack()

if exists("datasets/adversarial_datasets/MI-FGSMAttack/e4/dataset_spec.pb")==False:
    
    print("MI-FGSM with epsilon=4")
    MI_FGSM.generateDataset("MI-FGSMAttack",
                            clean_dataset, 
                            model_substitute, 
                            eps=4.0, 
                            eps_iter=4.0/20, 
                            nb_iter=20,
                            norm=np.inf,
                            spec="e4")

    print("MI-FGSM with epsilon=8")
    MI_FGSM.generateDataset("MI-FGSMAttack",
                            clean_dataset, 
                            model_substitute, 
                            eps=8.0, 
                            eps_iter=8.0/20, 
                            nb_iter=20,
                            norm=np.inf,
                            spec="e8")

    print("MI-FGSM with epsilon=16")
    MI_FGSM.generateDataset("MI-FGSMAttack",
                            clean_dataset, 
                            model_substitute, 
                            eps=16.0, 
                            eps_iter=16.0/20, 
                            nb_iter=20,
                            norm=np.inf,
                            spec="e16")

    print("MI-FGSM with epsilon=32")
    MI_FGSM.generateDataset("MI-FGSMAttack",
                            clean_dataset, 
                            model_substitute, 
                            eps=32.0, 
                            eps_iter=32.0/20, 
                            nb_iter=20,
                            norm=np.inf,
                            spec="e32")
else:
    print("MI-FGSM adversarial Dataset exists")
    
from attack_noise import NoiseAttack
Noise = NoiseAttack()
    
if exists("datasets/evaluation_noise/dataset_spec.pb")==False:
    print("Random noise dataset with epsilon=8")
    Noise.generateDataset(clean_dataset, 
                          model_substitute, 
                          eps=8.0)

## Phase 3: Evaluate (Adversarial) Datasets on Models

Phase 4 **-> Evaluation.ipynb**

## Phase 4: Generate Visualizations of Results

Phase 5 **-> Results.ipynb**